In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root_dir = '/content/drive/MyDrive/ML4BDS/Lab sessions/hackathon_demo/2024 gene exp'

### Define train dataset and split train-valid

In [ ]:
def read_fasta_as_string(fasta_filename):
    """
    Reads a FASTA file and returns the sequence as a string.

    Parameters:
    - fasta_filename (str): The name of the FASTA file to read.

    Returns:
    - str: The DNA sequence as a string.
    """
    sequence = ""
    with open(fasta_filename, 'r') as fasta_file:
        for line in fasta_file:
            # Skip the header line
            if line.startswith(">"):
                continue
            # Remove newline characters and concatenate
            sequence += line.strip()
    return sequence

In [ ]:
!pip install kipoiseq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 837.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 23.2.0
    Uninstalling attrs-23.2.0:
      Suc

In [ ]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from kipoiseq.transforms.functional import one_hot_dna

class ExpDataset(Dataset):
    """Dataset for reading fasta and exp."""

    def __init__(self, csv_file, fasta_dir='./fasta'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
        """
        # Read the CSV file
        self.data_frame = pd.read_csv(csv_file)
        self.fasta_dir = fasta_dir
        if self.data_frame.shape[1] == 1: # has target means has sequence in the training dataset
            self.has_target = False
        else:
            self.has_target = True

    def __len__(self):
        # Return the number of rows in the DataFrame
        return len(self.data_frame)

    def __getitem__(self, idx):
        sample = {}
        rec = self.data_frame.iloc[idx]
        seq_filename = self.fasta_dir + '/' + rec['SeqId'] + '.fa'
        sample['seq'] = torch.tensor(
            one_hot_dna( # one hot 0, 1 output
                read_fasta_as_string(
                    seq_filename)).astype(np.float32))
        if self.has_target:
            sample['target'] = torch.tensor(
                rec.iloc[1:].values.astype(np.float32))
        return sample

In [ ]:
train_ds = ExpDataset(root_dir +'/train_subset.csv', root_dir + '/fasta')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ML4BDS/Lab sessions/hackathon_demo/2024 gene exp/train_subset.csv'

In [ ]:
sample = train_ds[0]

In [ ]:
sample['seq']

In [ ]:
sample['target'][:20]

In [ ]:
sample['seq'].shape, sample['target'].shape

In [ ]:
from torch.utils.data import random_split

dataset_size = len(train_ds)
valid_size = int(dataset_size * 0.2)  # 20% for validation
train_size = dataset_size - valid_size  # Remaining for training

# Split the dataset
train_ds, valid_ds = random_split(train_ds, [train_size, valid_size])

In [ ]:
len(train_ds), len(valid_ds)

In [ ]:
from torch.utils.data import DataLoader
# Define the DataLoader
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
valid_dl = DataLoader(valid_ds, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

### Baseline model training

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class GeneExpressionCNN(nn.Module):
    def __init__(self):
        super(GeneExpressionCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=4, out_channels=128, kernel_size=10)
        self.pool = nn.AvgPool1d(kernel_size=900, stride=900)
        self.fc1 = nn.Linear(128 * 54, 500)
        self.fc2 = nn.Linear(500, 399)

    def forward(self, x):
        # Reshape the input to ensure it's [Batch, Channels, Length]
        # Assuming x is initially [Batch, Length, Channels]
        x = x.transpose(1, 2)  # Now x is [Batch, Channels, Length]
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor for the fully connected layer
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Model initialization
model = GeneExpressionCNN()
model

In [ ]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")

In [ ]:
import torch.optim as optim
model.to(device)  # Move the model to the GPU if available
# Loss and optimizer
criterion = nn.MSELoss() # can be changed, here is the simple MSE as the loss function
optimizer = optim.Adam(model.parameters(), lr=0.01)

**define model metric**

In [ ]:
def r_squared(y_true, y_pred):
    # Calculate the total sum of squares (SST)
    sst = torch.sum((y_true - torch.mean(y_true)) ** 2)
    # Calculate the residual sum of squares (SSR)
    ssr = torch.sum((y_true - y_pred) ** 2)
    # Calculate the R^2 score
    r2 = 1 - ssr / (sst + 1e-8)
    return r2

**define train and validate routines**

In [ ]:
from tqdm.notebook import tqdm

# Function to validate the model
def validate_model(dl):
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    val_metric = 0.0
    with torch.no_grad():  # No gradient computation
        for batch in dl:
            inputs, targets = batch['seq'], batch['target']
            inputs, targets = inputs.to(device), targets.to(device)  # Move data to GPU
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
            metric = r_squared(targets, outputs)
            val_metric += metric * inputs.size(0)

    val_loss /= len(dl.dataset)
    val_metric /= len(dl.dataset)
    print(f'Validation Loss: {val_loss:.4f}, Validation R^2: {val_metric:.4f}')
    return val_loss, val_metric


# Function to train the model
def train_model(num_epochs):
    model.train()  # Set the model to training mode
    best_metric = float('-inf')
    for epoch in range(num_epochs):
        running_loss = 0.0
        running_metric = 0.0
        for batch in tqdm(train_dl):
            inputs, targets = batch['seq'], batch['target']
            inputs, targets = inputs.to(device), targets.to(device)  # Move data to GPU
            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, targets)  # Compute the loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize

            running_loss += loss.item() * inputs.size(0)
            metric = r_squared(targets, outputs)
            running_metric += metric * inputs.size(0)

        epoch_loss = running_loss / len(train_dl.dataset)
        epoch_metric = running_metric / len(train_dl.dataset)
        print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}, R^2: {epoch_metric:.4f}')

        # Validation step
        _, val_metric = validate_model(valid_dl)
        if val_metric >= best_metric:
            torch.save(model.state_dict(), root_dir + '/hackathon_model_state.pth')
            best_metric = val_metric

def reset_weights(m):
    '''
    Try resetting model weights
    '''
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [ ]:
model.apply(reset_weights)
num_epochs = 5
train_model(num_epochs)

Reset trainable parameters of layer = Conv1d(4, 128, kernel_size=(10,), stride=(1,))
Reset trainable parameters of layer = Linear(in_features=6912, out_features=500, bias=True)
Reset trainable parameters of layer = Linear(in_features=500, out_features=399, bias=True)


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1, Loss: 9.4830, R^2: -0.8307
Validation Loss: 5.7199, Validation R^2: -0.0229


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 2, Loss: 5.2961, R^2: 0.0281
Validation Loss: 5.4009, Validation R^2: 0.0353


...

Epoch 3, Loss: 5.2851, R^2: 0.0258
Validation Loss: 5.4143, Validation R^2: 0.0318


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 4, Loss: 5.0386, R^2: 0.0744
Validation Loss: 5.4160, Validation R^2: 0.0308


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 5, Loss: 4.9087, R^2: 0.1008
Validation Loss: 4.9418, Validation R^2: 0.1173


### Make prediction on the test set for submission

In [ ]:
# Function to predict using the model
def predict_model(dl):
    model.eval()  # Set the model to evaluation mode
    out_pool = []
    with torch.no_grad():  # No gradient computation
        for batch in tqdm(dl):
            inputs = batch['seq']
            inputs = inputs.to(device)  # Move data to GPU
            outputs = model(inputs)
            out_pool.append(outputs.cpu())

    return torch.cat(out_pool)

In [ ]:
model.load_state_dict(torch.load(root_dir + '/hackathon_model_state.pth'))

In [ ]:
test_masked_ds = ExpDataset(root_dir +'/test_masked_subset.csv', root_dir + '/fasta')
test_masked_dl = DataLoader(test_masked_ds, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
test_out = predict_model(test_masked_dl)

  0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:
test_out.shape

torch.Size([1000, 399])

In [ ]:
test_out_df = pd.DataFrame(test_out.numpy(), columns=train_ds.dataset.data_frame.columns[1:])

In [ ]:
pred_df = pd.concat([test_masked_ds.data_frame, test_out_df], axis=1)

In [ ]:
pred_df.head()

,SeqId,E0001,E0002,E0003,E0004,E0005,E0006,E0007,E0008,E0009,...,E0390,E0391,E0392,E0393,E0394,E0395,E0396,E0397,E0398,E0399
0,S010036,2.476245,2.195931,2.092766,2.188402,2.043166,2.277982,2.170509,2.313032,2.166193,...,2.275849,2.338009,2.526620,2.760087,2.400112,2.527502,2.447612,2.623280,2.595776,2.595699
1,S012718,2.524069,2.239680,2.167720,2.252346,2.085926,2.324636,2.223694,2.384043,2.225045,...,2.353540,2.395438,2.584832,2.811758,2.451239,2.586977,2.514495,2.684041,2.652318,2.633865
2,S013672,2.145538,1.913915,1.901244,1.959914,1.793380,2.005386,1.918672,2.071490,1.921165,...,2.048819,2.053629,2.215088,2.392338,2.111267,2.212621,2.177767,2.302710,2.274477,2.223499
3,S011941,1.825693,1.634648,1.660743,1.698981,1.541724,1.742002,1.641717,1.793798,1.663887,...,1.747442,1.782033,1.917051,2.040596,1.835114,1.887413,1.860928,1.999625,1.950174,1.910848
4,S000275,1.570977,1.394076,1.405951,1.432898,1.307377,1.500174,1.389935,1.516733,1.420590,...,1.439565,1.539228,1.662815,1.764049,1.587898,1.610432,1.558945,1.741117,1.666792,1.683164
